In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

def getOpticalFlow(video):
    # initialize the list of optical flows
    gray_video = []
    for i in range(len(video)):
        img = cv2.cvtColor(video[i], cv2.COLOR_RGB2GRAY)
        gray_video.append(np.reshape(img, (224, 224, 1)))

    flows = []
    for i in range(0, len(video) - 1):
        # calculate optical flow between each pair of frames
        flow = cv2.calcOpticalFlowFarneback(gray_video[i], gray_video[i + 1], None, 0.5, 3, 15, 3, 5, 1.2,
                                            cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        # subtract the mean in order to eliminate the movement of camera
        flow[..., 0] -= np.mean(flow[..., 0])
        flow[..., 1] -= np.mean(flow[..., 1])
        # normalize each component in optical flow
        flow[..., 0] = cv2.normalize(flow[..., 0], None, 0, 255, cv2.NORM_MINMAX)
        flow[..., 1] = cv2.normalize(flow[..., 1], None, 0, 255, cv2.NORM_MINMAX)
        # Add into list
        flows.append(flow)

    # Padding the last frame as empty array
    flows.append(np.zeros((224, 224, 2)))

    return np.array(flows, dtype=np.float32)


def farneback_visual(flows, file_path):
    # visualization farneback optical flow map
    # save the map as 'farneback_optical_flow.mp4'
    h, w = flows.shape[1:3]

    path = '/content/drive/MyDrive/solution_challange/examples'
    file_name = '/'.join(file_path.split('.')[0].split('/')[-3:]) + '_opticalFlow'

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out_video = cv2.VideoWriter(os.path.join(path, file_name+'.mp4'), fourcc, 30.0, (int(w), int(h)), isColor=True)

    hsv = np.zeros((h, w, 3))
    hsv[..., 1] = 255
    for flow in flows:
        mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        hsv[...,0] = ang*180/np.pi/2
        hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
        hsv = np.float32(hsv)
        rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
        out_video.write(np.uint8(rgb))
    out_video.release()
    return None
    pass


def Video2Npy(file_path, resize=(224,224)):

    # Load video
    cap = cv2.VideoCapture(file_path)
    # Get number of frames
    len_frames = int(cap.get(7))
    # Extract frames from video
    try:
        frames = []
        for i in range(len_frames-1):
            _, frame = cap.read()
            frame = cv2.resize(frame,resize, interpolation=cv2.INTER_AREA)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = np.reshape(frame, (224,224,3))
            frames.append(frame)
    except:
        print("Error: ", file_path, len_frames,i)
    finally:
        frames = np.array(frames)
        cap.release()

    # Get the optical flow of video
    flows = getOpticalFlow(frames)

    # Visualize optical flow map
    optical_flow_map = farneback_visual(flows, file_path)

    result = np.zeros((len(flows),224,224,5))
    result[...,:3] = frames
    result[...,3:] = flows

    return result


def Save2Npy(file_dir, save_dir):   # modify this code to save the npy files for your directory or path
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    # List the files
    videos = os.listdir(file_dir)
    for v in tqdm(videos):
        # Split video name
        video_name = v.split('.')[0]
        # Get src
        video_path = os.path.join(file_dir, v)
        # Get dest
        save_path = os.path.join(save_dir, video_name+'.npy')
        # Load and preprocess video
        data = Video2Npy(file_path=video_path, resize=(224,224))
        data = np.uint8(data)
        # Save as .npy file
        np.save(save_path, data)

    return None

### convert data and save it (offline)

In [ ]:
source_path = './Data/examples'
target_path = './Data/npy_video_ex'

for f1 in ['train', 'val']:
    for f2 in ['Fight', 'NonFight']:
        path1 = os.path.join(source_path, f1, f2)
        path2 = os.path.join(target_path, f1, f2)
        Save2Npy(file_dir=path1, save_dir=path2)

100%|██████████| 100/100 [07:45<00:00,  4.65s/it]


## **1. Build Data Loader**

In [ ]:
import torch
import torch.utils.data as data
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os
import cv2

class DataGenerator(Dataset):

    def __init__(self, directory, data_augmentation=True, phase='train'):
        self.phase=phase
        self.directory = directory
        self.data_aug = data_augmentation
        self.X_path, self.Y_dict = self.search_data()
        self.print_stats()

    def __len__(self):
        steps_per_epoch = int(len(self.X_path))
        return steps_per_epoch

    def __getitem__(self, index):
        data, label = self.data_generation(self.X_path[index])
        return data.float(), label

    def load_data(self, path):
        data = np.load(path, mmap_mode='r')
        data = self.uniform_sampling(data, target_frames=64)
        if self.data_aug:
            data[..., :3] = self.color_jitter(data[..., :3])
            data = self.random_flip(data, prob=0.5)
        data[..., :3] = self.normalize(data[..., :3]) # Normalize RGB
        data[..., 3:] = self.normalize(data[..., 3:]) # Normalize optical flows
        return data

    # shuffle data
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    # Normalize data
    def normalize(self, data):
        mean = data.mean()
        std = data.std()
        return (data - mean) / std

    def random_flip(self, video, prob):
        s = np.random.rand()
        if s < prob:
            video = np.flip(video, (2,)) # Flip in width direction
        return video

    def uniform_sampling(self, video, target_frames=64):
        # get total frames of input video and calculate sampling interval
        len_frames = int(len(video))
        interval = int(np.ceil(len_frames/target_frames))
        # init empty list for sampled video and
        sampled_video = []
        for i in range(0,len_frames,interval):
            sampled_video.append(video[i])
        # calculate numer of padded frames and fix it
        num_pad = target_frames - len(sampled_video)
        padding = []
        if num_pad>0:
            for i in range(-num_pad,0):
                try:
                    padding.append(video[i])
                except:
                    padding.append(video[0])
            sampled_video += padding # Add padding results
        # get sampled video
        return np.array(sampled_video, dtype=np.float32)

    # Jitter = spread values
    def color_jitter(self, video):
        s_jitter = np.random.uniform(-0.2, 0.2)
        v_jitter = np.random.uniform(-30, 30)
        for i in range(len(video)):
            hsv = cv2.cvtColor(np.array(video[i]), cv2.COLOR_RGB2HSV) # Convert RGB -> HSV
            s = hsv[..., 1] + s_jitter # saturation jitter
            v = hsv[..., 2] + v_jitter # Value jitter
            # Flip
            s[s < 0] = 0
            s[s > 1] = 1
            v[v < 0] = 0
            v[v > 255] = 255
            hsv[..., 1] = s # set jittered saturation
            hsv[..., 2] = v # Set jittered value
            video[i] = torch.Tensor(cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)) # Convert HSV -> RGB again
        return video

    # print current state sof the Dataset class
    def print_stats(self):
            self.n_files = len(self.X_path)
            self.n_classes = len(self.dirs)
            self.indexes = np.arange(len(self.X_path))
            np.random.shuffle(self.indexes)
            print("Found {} files belonging to {} classes.".format(self.n_files, self.n_classes))
            for i, label in enumerate(self.dirs):
                print('{:10s} : {}'.format(label, i))

    def search_data(self):
        X_path = []
        Y_dict = {}
        self.dirs = sorted(os.listdir(self.directory)) # Get sorted file directories
        #one_hots = [0.0, 0.0]
        for i, folder in enumerate(self.dirs):
            folder_path = os.path.join(self.directory, folder)
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                # Add file path into X_path
                X_path.append(file_path)
                # Assign one hot encoded vector into Y_dict
                Y_dict[file_path] = i
        return X_path, Y_dict

    # define batch x using 'load_data' function and batch_y from self.Y_dict.
    def data_generation(self, batch_path):
        batch_x = np.array(self.load_data(batch_path))
        batch_y = np.array(self.Y_dict[batch_path])

        batch_x = torch.tensor(batch_x)
        batch_y = torch.tensor(batch_y)

        return batch_x, batch_y

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        self.relu=nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.maxpool = nn.MaxPool3d(kernel_size=(8, 1 ,1), stride=(8, 1, 1))

        # Construct block of RGB layers which takes RGB channel(3) as input
        self.rgb_conv1 = nn.Conv3d(3, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.rgb_conv2 = nn.Conv3d(16, 16, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.rgb_maxpool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.rgb_conv3 = nn.Conv3d(16, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.rgb_conv4 = nn.Conv3d(16, 16, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.rgb_maxpool2 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.rgb_conv5 = nn.Conv3d(16, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.rgb_conv6 = nn.Conv3d(32, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.rgb_maxpool3 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.rgb_conv7 = nn.Conv3d(32, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.rgb_conv8 = nn.Conv3d(32, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.rgb_maxpool4 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        # Construct block of optical flow layers which takes the optical flow channel(2) as input
        self.opt_conv1 = nn.Conv3d(2, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.opt_conv2 = nn.Conv3d(16, 16, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.opt_maxpool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.opt_conv3 = nn.Conv3d(16, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.opt_conv4 = nn.Conv3d(16, 16, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.opt_maxpool2 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.opt_conv5 = nn.Conv3d(16, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.opt_conv6 = nn.Conv3d(32, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.opt_maxpool3 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.opt_conv7 = nn.Conv3d(32, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.opt_conv8 = nn.Conv3d(32, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.opt_maxpool4 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        # Construct merging Block
        self.merge_conv1 = nn.Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.merge_conv2 = nn.Conv3d(64, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.merge_maxpool1 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.merge_conv3 = nn.Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.merge_conv4 = nn.Conv3d(64, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.merge_maxpool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.merge_conv5 = nn.Conv3d(64, 128, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.merge_conv6 = nn.Conv3d(128, 128, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.merge_maxpool3 = nn.MaxPool3d(kernel_size=(2, 3, 3), stride=(2, 3, 3))

        # Fully Connected Layers
        self.fc1 = nn.Linear(128, 128)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 2)

        # Initialize weights
        self.__init_weight()

    def forward(self, x):
        x = x.transpose(2, 4)
        x = x.transpose(3, 4)
        x = x.transpose(1, 2)
        rgb = x[:,:3,:,:,:]
        opt = x[:,3:5,:,:,:]

        # Pass through the RGB data through the blocks of RGB layers
        rgb = self.rgb_conv1(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_conv2(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_maxpool1(rgb)
        rgb = self.rgb_conv3(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_conv4(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_maxpool2(rgb)

        rgb = self.rgb_conv5(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_conv6(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_maxpool3(rgb)
        rgb = self.rgb_conv7(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_conv8(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_maxpool4(rgb)

        # Pass through the optical flow data through the blocks of RGB layers
        opt = self.opt_conv1(opt)
        opt = self.relu(opt)
        opt = self.opt_conv2(opt)
        opt = self.relu(opt)
        opt = self.opt_maxpool1(opt)
        opt = self.opt_conv3(opt)
        opt = self.relu(opt)
        opt = self.opt_conv4(opt)
        opt = self.relu(opt)
        opt = self.opt_maxpool2(opt)

        opt = self.opt_conv5(opt)
        opt = self.relu(opt)
        opt = self.opt_conv6(opt)
        opt = self.relu(opt)
        opt = self.opt_maxpool3(opt)
        opt = self.opt_conv7(opt)
        opt = self.sigmoid(opt)
        opt = self.opt_conv8(opt)
        opt = self.sigmoid(opt)
        opt = self.opt_maxpool4(opt)

        # Fuse by performing elementwise multiplication of rgb and opt tensors.
        fused = rgb * opt
        # Perform maxpooling of fused
        fused = self.maxpool(fused)

        # Pass through the fused data into merging block
        merged = self.merge_conv1(fused)
        merged = self.relu(merged)
        merged = self.merge_conv2(merged)
        merged = self.relu(merged)
        merged = self.merge_maxpool1(merged)
        merged = self.merge_conv3(merged)
        merged = self.relu(merged)
        merged = self.merge_conv4(merged)
        merged = self.relu(merged)
        merged = self.merge_maxpool2(merged)
        merged = self.merge_conv5(merged)
        merged = self.relu(merged)
        merged = self.merge_conv6(merged)
        merged = self.relu(merged)
        merged = self.merge_maxpool3(merged)

        # Fully Connected Layers
        x = merged.view(merged.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)

        return x

    def __init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                # Perform weight initialization ("kaiming normal")
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    #nn.init.constant_(m.bias, 0)
                    fan_in, _ = nn.init._calculate_fan_in_and_fan_out(m.weight)
                    bound = 1 / torch.sqrt(torch.tensor(fan_in))
                    nn.init.uniform_(m.bias, -bound, bound)

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.8/257.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


##  Training the Model

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from tqdm import tqdm
import wandb


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define model, optimizer and criterion
model = FusionModel().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-6, nesterov=True)
scheduler = StepLR(optimizer, step_size=10, gamma=0.7)
loss_fn = nn.CrossEntropyLoss()

trainset_path = './Data/npy_video_ex/train'
validation_path = './Data/npy_video_ex/val'

# define dataset and dataloader
train_dataset = DataGenerator(trainset_path, data_augmentation=True, phase='train')
val_dataset = DataGenerator(validation_path, data_augmentation=True, phase='val')
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

epochs = 10
min_loss = np.inf


Found 400 files belonging to 2 classes.
Fight      : 0
NonFight   : 1
Found 40 files belonging to 2 classes.
Fight      : 0
NonFight   : 1


In [ ]:
from tqdm.auto import tqdm as tqdm1
import torch.nn.functional as F
def train():
    model.train()
    acc_temp = 0
    running_loss = 0.0

    # forward propagation and backpropagation
    # calculate accuracy and loss on training set
    for inputs, targets in tqdm1(train_loader, desc='Training'):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        acc_temp += (predicted == targets).sum().item()
        running_loss += loss.item()

    train_acc = acc_temp / len(train_loader.dataset)
    train_loss = running_loss / len(train_loader.dataset)

    return train_acc, train_loss

In [ ]:
def val():
    model.eval()
    with torch.no_grad():
        running_loss_val = 0
        acc_temp_val =0

        # calculate accuracy and loss on validation set
        for inputs, targets in tqdm1(val_loader, desc='Validation'):
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, targets)

            _, predicted = torch.max(outputs, 1)
            acc_temp_val += (predicted == targets).sum().item()
            running_loss_val += loss.item()

        val_acc = acc_temp_val / len(val_loader.dataset)
        val_loss = running_loss_val / len(val_loader.dataset)

    return val_acc, val_loss

In [ ]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import random
import numpy as np

seed=0
random.seed(seed)
np.random.seed(seed)
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device =='cuda':
    print('gpu device is using')
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False

wandb.login()
wandb.init(project='solution_challange', name='1')

for epoch in tqdm(range(epochs)):
    # Perform training and validation
    # Save the weights
    # Log the training and validation results
    epoch_acc, epoch_loss = train()
    epoch_acc_val, epoch_loss_val = val()

    print(f'Epoch: {epoch}, Train Accuracy: {epoch_acc}, Train Loss: {epoch_loss}')
    print(f'Epoch: {epoch}, Valid Accuracy: {epoch_acc_val}, Valid Loss: {epoch_loss_val}')
    wandb.log({'Epoch':epoch, 'Train Accuracy':epoch_acc, "Train Loss":epoch_loss})
    wandb.log({'Epoch':epoch, 'Valid Accuracy':epoch_acc_val, "Valid Loss":epoch_loss_val})

    if epoch_loss_val <= min_loss:
        min_loss = epoch_loss_val
        torch.save(model.state_dict(), './Output/model_weight3.pth')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chanwon0721 (compute_vision). Use `wandb login --relogin` to force relogin


  0%|          | 0/10 [00:00<?, ?it/s]

Training:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
source_path = './Data/examples'
target_path = './Data/npy_video_ex'

for f1 in ['test']:
    for f2 in ['Fight', 'NonFight']:
        path1 = os.path.join(source_path, f1, f2)
        path2 = os.path.join(target_path, f1, f2)
        Save2Npy(file_dir=path1, save_dir=path2)

100%|██████████| 10/10 [00:49<00:00,  4.97s/it]


In [ ]:
best_model = './Data/model_weight3.pth'
test_path = './Data/npy_video_ex/test'

test_dataset = DataGenerator(directory=test_path, data_augmentation=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=0)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

model.load_state_dict(torch.load(best_model))
model.eval()
with torch.no_grad():
    # calculate accuracy and AUROC on test set
    running_loss_test = 0
    acc_temp_test =0

    preds = []
    targets = []
    for input, target in tqdm(test_loader, desc='Test'):
        input, target = input.to(device), target.to(device)

        output = model(input)
        loss = loss_fn(output, target)

        _, predicted = torch.max(output, 1)
        acc_temp_test += (predicted == target).sum().item()
        running_loss_test += loss.item()

        preds.append(predicted.cpu().numpy())
        targets.append(target.cpu().numpy())

    test_acc = acc_temp_test / len(test_loader.dataset)
    test_loss = running_loss_test / len(test_loader.dataset)
    preds = np.array(preds).flatten()
    targets = np.array(targets).flatten()

print(f'test accuracy: {test_acc}')
print(f'AUROC: {roc_auc_score(targets, preds)}')

fpr, tpr, thresholds = roc_curve(targets, preds)
plt.plot(fpr, tpr)
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.show()